In [ ]:
pip install nltk

In [ ]:
pip install python-docx

In [ ]:
pip install transformers sentencepiece

In [ ]:
pip install rouge

In [ ]:
pip install pythainlp

In [ ]:
import nltk
import numpy as np
import networkx as nx
import pandas as pd
import docx
from docx.enum.text import WD_COLOR_INDEX
from nltk.cluster.util import cosine_distance
from transformers import AutoTokenizer, AutoModel
from pythainlp.tokenize import word_tokenize
import os
from rouge import Rouge
import sys
sys.setrecursionlimit(10000)  # หรือปรับตัวเลขตามที่เหมาะสม

In [ ]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def highlight_sentences(text, sum_list):
    doc = docx.Document()
    doc.add_heading('Summarized Text', 0)
    para = doc.add_paragraph('')
    sentences = text.split('\n')
    for st in sentences:
        if any(keyword in st for keyword in sum_list):
            para.add_run(st + '\n').font.highlight_color = WD_COLOR_INDEX.YELLOW
        else:
            para.add_run(st + '\n')
    return doc

In [ ]:
def process_file(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            corpus = file.read().splitlines()

        sentences = [s.strip() for s in corpus]
        sentence_size = len(sentences)
        if sentence_size == 0:
            print(f"File {file_name} is empty or contains no valid sentences.")
            return []

        # Tokenize and encode sentences
        sentences_tokens = [tokenizer(s, return_tensors="pt") for s in sentences]
        sentences_encode = [model(**token) for token in sentences_tokens]
        sentences_vec = [np.squeeze(vec[1].detach().numpy()) for vec in sentences_encode]

        # Build sentence similarity matrix
        def build_similarity_matrix(sentences):
            similarity_matrix = np.zeros((len(sentences), len(sentences)))
            for idx1 in range(len(sentences)):
                for idx2 in range(len(sentences)):
                    if idx1 == idx2:
                        continue
                    similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2])
            return similarity_matrix

        def sentence_similarity(sent1, sent2):
            return 1 - cosine_distance(sent1, sent2)

        sentence_similarity_matrix = build_similarity_matrix(sentences_vec)
        sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
        scores = nx.pagerank(sentence_similarity_graph)

        # Rank sentences by score
        ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
        top_n = round(sentence_size * 0.2)
        sum_list = [ranked_sentence[i][1] for i in range(top_n)]

        # Highlight sentences in a Word document
        doc = highlight_sentences('\n'.join(sentences), sum_list)
        output_file = f"{os.path.splitext(file_name)[0]}_highlighted.docx"
        doc.save(output_file)
        print(f"Processed and saved: {output_file}")
        return sum_list
    except FileNotFoundError:
        print(f"File not found: {file_name}")
        return []
    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        return []

In [ ]:
highlighted_file = 'clip_13.txt'
sum_list = process_file(highlighted_file)

Processed and saved: clip_13_highlighted.docx


In [ ]:
human_summaries = [f"clip_13_{i}.txt" for i in range(1, 5)]

In [ ]:
def tokenizer(st_list):
    words = []
    for s in st_list:
        w = word_tokenize(s)
        w = [temp for temp in w if temp != ' ']
        if w != []:
            words.append(' '.join(w).lower())
    return words

In [ ]:
def evaluate_summary(y_test, predicted):
    rouge = Rouge()
    scores = rouge.get_scores(y_test, predicted, avg=True)
    score_1 = round(scores['rouge-1']['f'], 2)
    score_2 = round(scores['rouge-2']['f'], 2)
    score_L = round(scores['rouge-l']['f'], 2)
    print('Rouge1:', score_1)
    print('Rouge2:', score_2)
    print('RougeL:', score_L)
    print('Avg Rouge:', round(np.mean([score_1, score_2, score_L]), 2))


In [ ]:
for human_summarie_file in human_summaries:
    try:
        with open(human_summarie_file, 'r', encoding='utf-8') as file:
            human_summarie = file.read().splitlines()

        # Tokenize the human summary and the highlighted sentences
        human_summarie_word = tokenizer(human_summarie)
        sum_list_word = tokenizer(sum_list)

        # Convert lists to string for ROUGE comparison
        test = ' '.join(human_summarie_word)  # Human summary
        predicted = ' '.join(sum_list_word)  # Program summary

        # Print for verification
        print(f"\nComparing with {human_summarie_file}...")
        print("Test (Human summary):", test)
        print("Predicted (Highlighted summary):", predicted)

        # Calculate ROUGE score
        evaluate_summary(test, predicted)

    except FileNotFoundError:
        print(f"File not found: {human_summarie_file}")
    except Exception as e:
        print(f"Error processing {human_summarie_file}: {e}")


Comparing with clip_13_1.txt...
Test (Human summary): ตอนนี้ นะ ครับ หรือ organization ของ เรา องค์ เรา เปลี่ยน ออง กร เรา เปลี่ยน หมาย ถึงว่า เปลี่ยน รูปแบบ บริหาร เปลี่ยน ของ มัน ไป เลย แต่ เดี๋ยวนี้ โลก มัน เปลี่ยนไป เรา จะ ใช้ server หลาย หลาย ตัว ทำ cluster ถ้า ใคร เรียน ผู้ใช้ software ขึ้น นะ enterprise เรียน rip-off เดี๋ยว พอ เข้าใจ ถ้า ใคร ยัง ไม่ ลอง เรียน เรา ไปดู นะคะ ก็ ใช้ server เป็น ภาพ หลาย ตัว แต่ server ไม่ ได้ แพง มาก แต่ มัน สามารถ จะ ขยาย เขา เรียก ว่า scale outscale out ก็ คือ ใช้ เกิน หลาย ๆ ตัวต่อ ตอน medical desk หลังคา หรือ เป็น cluster ใหญ่ ขึ้น มา ได้ ดู ที่ เรา ต้องการ ต่อมา นะคะ ตัว application และ system เปลี่ยน ระบบ สิ่ง ต้อง ต้องการ ของ ลูกค้า เปลี่ยนไป เมื่อกี้ บอก แล้ว แล้ว ต้อง ว่า local นะคะ แทนที่ มี ลูกค้า แค่ ใน องค์ ของ เรา หรือ คนใช้ ใน องค์ ของ เรา เรา ก็ ต้อง ใช้ อีก อ่อน หลาย ๆ ที่ dataway ต้อง เปลี่ยน ชนิด ของ dataway ก็ เปลี่ยน สมุด ว่า เรา ใช้ oracle อยู่ และ 8,000 คน ต่อ 8,000 ลูก ค้าน ตาย ชัน ต่อ 1 ครั้ง ตอนนี้ มัน ไม่ ไหว แล้ว ลูกค้า

In [28]:
a = {"Rouge1": 0.69, "Rouge2": 0.56, "RougeL": 0.58, "Avg Rouge": 0.61}
b = {"Rouge1": 0.67, "Rouge2": 0.53, "RougeL": 0.61, "Avg Rouge": 0.6}
c = {"Rouge1": 0.66, "Rouge2": 0.54, "RougeL": 0.54, "Avg Rouge": 0.58}
d = {"Rouge1": 0.39, "Rouge2": 0.12, "RougeL": 0.22, "Avg Rouge": 0.24}

In [29]:
results = [a, b, c, d]

In [31]:
avg_rouge1 = sum([result["Rouge1"] for result in results]) / len(results)
avg_rouge2 = sum([result["Rouge2"] for result in results]) / len(results)
avg_rougeL = sum([result["RougeL"] for result in results]) / len(results)
avg_rouge = sum([result["Avg Rouge"] for result in results]) / len(results)

print(f"ค่าเฉลี่ยของ Rouge1 จากการทดสอบ 4 ชุดคือ: {avg_rouge1}")
print(f"ค่าเฉลี่ยของ Rouge2 จากการทดสอบ 4 ชุดคือ: {avg_rouge2}")
print(f"ค่าเฉลี่ยของ RougeL จากการทดสอบ 4 ชุดคือ: {avg_rougeL}")
print(f"ค่าเฉลี่ยของ Avg Rouge จากการทดสอบ 4 ชุดคือ: {avg_rouge}")

ค่าเฉลี่ยของ Rouge1 จากการทดสอบ 4 ชุดคือ: 0.6025
ค่าเฉลี่ยของ Rouge2 จากการทดสอบ 4 ชุดคือ: 0.4375
ค่าเฉลี่ยของ RougeL จากการทดสอบ 4 ชุดคือ: 0.4875
ค่าเฉลี่ยของ Avg Rouge จากการทดสอบ 4 ชุดคือ: 0.5075000000000001


In [32]:
clip_1 = {"Rouge1": 0.6025, "Rouge2": 0.4375, "RougeL": 0.4875, "Avg Rouge": 0.5075}
clip_2 = {"Rouge1": 0.56, "Rouge2": 0.3875, "RougeL": 0.315, "Avg Rouge": 0.4225}
clip_3 = {"Rouge1": 0.645, "Rouge2": 0.385, "RougeL": 0.4675, "Avg Rouge": 0.4975}
clip_5 = {"Rouge1": 0.47, "Rouge2": 0.2345, "RougeL": 0.2475, "Avg Rouge": 0.32}
clip_6 = {"Rouge1": 0.53, "Rouge2": 0.2925, "RougeL": 0.37, "Avg Rouge": 0.3975}
clip_7 = {"Rouge1": 0.5, "Rouge2": 0.33, "RougeL": 0.205, "Avg Rouge": 0.345}
clip_9 = {"Rouge1": 0.5925, "Rouge2": 0.455, "RougeL": 0.36, "Avg Rouge": 0.47}
clip_10 = {"Rouge1": 0.61, "Rouge2": 0.375, "RougeL": 0.325, "Avg Rouge": 0.4375}
clip_12 = {"Rouge1": 0.455, "Rouge2": 0.31, "RougeL": 0.3775, "Avg Rouge": 0.38}
clip_13 = {"Rouge1": 0.5175, "Rouge2": 0.37, "RougeL": 0.3325, "Avg Rouge": 0.43}

In [33]:
results = [clip_1, clip_2, clip_3, clip_5, clip_6, clip_7, clip_9, clip_10, clip_12, clip_13]

In [34]:
avg_rouge1 = sum([result["Rouge1"] for result in results]) / len(results)
avg_rouge2 = sum([result["Rouge2"] for result in results]) / len(results)
avg_rougeL = sum([result["RougeL"] for result in results]) / len(results)
avg_rouge = sum([result["Avg Rouge"] for result in results]) / len(results)

print(f"ค่าเฉลี่ยของ Rouge1 จากการทดสอบ 10 ชุดคือ: {avg_rouge1}")
print(f"ค่าเฉลี่ยของ Rouge2 จากการทดสอบ 10 ชุดคือ: {avg_rouge2}")
print(f"ค่าเฉลี่ยของ RougeL จากการทดสอบ 10 ชุดคือ: {avg_rougeL}")
print(f"ค่าเฉลี่ยของ Avg Rouge จากการทดสอบ 10 ชุดคือ: {avg_rouge}")

ค่าเฉลี่ยของ Rouge1 จากการทดสอบ 10 ชุดคือ: 0.5482500000000001
ค่าเฉลี่ยของ Rouge2 จากการทดสอบ 10 ชุดคือ: 0.3577
ค่าเฉลี่ยของ RougeL จากการทดสอบ 10 ชุดคือ: 0.34875
ค่าเฉลี่ยของ Avg Rouge จากการทดสอบ 10 ชุดคือ: 0.42074999999999996
